In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
import pickle

In [6]:
df=pd.read_csv('F:\\Generative AI Course With Langchain and Huggingface\\13 - ANN Project Implementation\\annclassification\\Churn_Modelling.csv')

In [7]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
# The row number, customerID, Surname aren't relevant for the model so we need to delete it
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
# we now convert the categgorical features to numeric
# Since there is a cordial relationship b/w M and F in Gender we use LabelEncoding
le=LabelEncoder()
df['Gender']=le.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [10]:
# Now we need to perform OHE on Geography as there is no cordial relationship b/w its data
ohe=OneHotEncoder()
geo=ohe.fit_transform(df[['Geography']])
geograph=pd.DataFrame(geo.toarray(),columns=ohe.get_feature_names_out(['Geography']))
geograph

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
# Appending the dataframe
data=pd.concat([df.drop('Geography',axis=1),geograph],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
# we now save the encoder in a pickle file
with open('labelencoder.pkl','wb') as file:
    pickle.dump(le,file)
    
with open('onehotencoder.pkl','wb') as file:
    pickle.dump(ohe,file)
    

In [13]:
X=data.drop('Exited',axis=1)
y=data["Exited"]
data.shape

(10000, 13)

In [14]:
#train-test-split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)
scalar=StandardScaler()
X_train=scalar.fit_transform(X_train)
X_test=scalar.transform(X_test)

In [15]:
# pickling scalar file


with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file)

In [16]:
# ANN Implementation

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [18]:
(X.shape[1],)

(12,)

In [19]:
# Build your own ANN model

model=Sequential(
[
Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ##HL1 layer
Dense(32,activation='relu'), #HL2 Layer
Dense(1,activation='sigmoid')
] ##output layer
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
import datetime
# Setting up the tensor board
log_dir="log/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
# Setting uup early stopping
earlystopping_check=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [22]:
history=model.fit(
X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,earlystopping_check]
)

Epoch 1/100


235/235 [==============================] - 4s 8ms/step - loss: 0.4725 - accuracy: 0.7907 - val_loss: 0.4076 - val_accuracy: 0.8268
Epoch 2/100
235/235 [==============================] - 1s 6ms/step - loss: 0.3959 - accuracy: 0.8336 - val_loss: 0.3694 - val_accuracy: 0.8484
Epoch 3/100
235/235 [==============================] - 1s 5ms/step - loss: 0.3630 - accuracy: 0.8528 - val_loss: 0.3553 - val_accuracy: 0.8628
Epoch 4/100
235/235 [==============================] - 1s 6ms/step - loss: 0.3479 - accuracy: 0.8579 - val_loss: 0.3490 - val_accuracy: 0.8608
Epoch 5/100
235/235 [==============================] - 2s 7ms/step - loss: 0.3393 - accuracy: 0.8615 - val_loss: 0.3425 - val_accuracy: 0.8624
Epoch 6/100
235/235 [==============================] - 2s 7ms/step - loss: 0.3345 - accuracy: 0.8608 - val_loss: 0.3476 - val_accuracy: 0.8604
Epoch 7/100
235/235 [==============================] - 2s 7ms/step - loss: 0.3315 - accuracy: 0.8619 - val_loss: 0.3457 - val_accuracy: 0.85

In [23]:
model.save('model.h5')

f:\Generative AI Course With Langchain and Huggingface\13 - ANN Project Implementation\ANNProject\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
# Load external Dashboard
%load_ext tensorboard

In [26]:
%tensorboard --logdir log/fit/20250209-203634

Reusing TensorBoard on port 6006 (pid 1916), started 0:00:31 ago. (Use '!kill 1916' to kill it.)

In [27]:
# Prediction

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd
import numpy as np

In [29]:
# load the model
model=load_model('model.h5')

In [2]:
# Loading the encoder and decoder
with open('onehotencoder.pkl','rb') as file:
    ohe=pickle.load(file)
with open('labelencoder.pkl','rb') as file:
    lbe=pickle.load(file)

In [6]:
ohe.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [3]:
lbe.classes_

array(['Female', 'Male'], dtype=object)

In [9]:
#lbe.transform(data['Gender'])[0]

1

In [4]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

data=pd.DataFrame([input_data])

In [5]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [33]:
data['Gender']=lbe.transform(data['Gender'])
ab=ohe.transform([data['Geography']]).toarray()
cd=ohe.get_feature_names_out(['Geography'])
geo=pd.DataFrame(ab,columns=cd)
data=pd.concat([data.drop('Geography',axis=1),geo],axis=1)
data

f:\Generative AI Course With Langchain and Huggingface\13 - ANN Project Implementation\ANNProject\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [35]:
# load the standards scalar

with open('scalar.pkl','rb') as file:
    scalar=pickle.load(file)
data_scaled=scalar.transform(data)
data_scaled

array([[-0.54007722,  0.91186722,  0.10685843, -0.69164931, -0.26034447,
         0.81311138,  0.64946204,  0.96978946, -0.88017893,  1.00053348,
        -0.57776083, -0.57735027]])

In [36]:
predict_churn=model.predict(data_scaled)
predict_churn


1/1 [==============================] - 0s 217ms/step


array([[0.02826444]], dtype=float32)

In [37]:
if predict_churn[0][0]>0.5:
    print("Customer most likely to exit")
else:
    print("Customer not likely to exit")

Customer not likely to exit
